# Model to ICEWS Events

In this analysis, I'm going to build a basic BDM model for the world based on COW alliance taus (with position = similarity to United States, to capture unipolarity) and see how predicted conflicts align with ICEWS events.

In [1]:
import copy
from collections import defaultdict, namedtuple, Counter
from itertools import combinations, permutations


import numpy as np
import scipy.stats

import pandas as pd
import networkx as nx

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from negotiation_model import *
from bdm_agent import *
from bdm_replication import *

In [3]:
import ipyparallel

# Setup model variant

In [4]:
class BDM_Agent_wo_Movement(Real_BDM_Agent):
    def lose_conflict(self, winner):
        pass

class BDMActor(NegotiationActor):
    DecisionClass = BDM_Agent_wo_Movement


In [13]:
class Model_Output:
    def __init__(self, model):
        '''
        Store data from model run.
        '''
        self.agent_vars = model.datacollector.get_agent_vars_dataframe()
        self.model_vars = model.datacollector.get_model_vars_dataframe()
        self.log = model.log

# Load data

(generated in 6.01)

In [5]:
agents = pd.read_csv("ICEWS_2004_agents.csv")

In [6]:
agents.head()

,Name,Position,Capability
0,Algeria,0.006786,0.005054
1,Argentina,0.731829,0.004797
2,Australia,0.192204,0.006817
3,Bangladesh,NaN,0.007891
4,Belgium,0.547674,0.004747


# Setup cluster

In [7]:
clients = ipyparallel.Client()
clients.ids

[0, 1, 2, 3]

In [8]:
dview = clients[:]

In [9]:
with dview.sync_imports():
    import random
    #from bdm_replication import BDMActor, Real_Negotiation_Model
    from bdm_replication import Real_Negotiation_Model

importing random on engine(s)
importing Real_Negotiation_Model from bdm_replication on engine(s)


In [16]:
dview.push({"agent_df": agents,
            "BDM_Agent_wo_Movement": BDM_Agent_wo_Movement,
           "BDMActor": BDMActor,
           "Model_Output": Model_Output})

<AsyncResult: finished>

In [17]:
def run_model_step(n):
    # Assemble model
    agents = []
    for i, row in agent_df.iterrows():
        name = row.Name
        capability = row.Capability
        position = row.Position
        salience = random.random()
        if not position.is_integer():
            position = 0.5 + random.normalvariate(0, 0.1)
        new_agent = BDMActor(name, capability, position, salience)

        new_agent.decision_model.Q = 0.5
        new_agent.decision_model.T = 0.5
        agents.append(new_agent)
    model = Real_Negotiation_Model(agents)
    
    # Run Model
    for j in range(24):
        model.step()
    return Model_Output(model)

# Run

In [18]:
model_results = dview.map_sync(run_model_step, range(4))

In [ ]:
class Model_Output:
    def __init__(self, model):
        '''
        Store data from model run.
        '''
        self.agent_vars = model.datacollector.get_agent_vars_dataframe()
        self.model_vars = model.datacollector.get_model_vars_dataframe()
        self.log = model.log

In [ ]:
all_models = [Model_Output(m) for m in model_results]

In [ ]:
df = model_results[-1].datacollector.get_model_vars_dataframe()
fig, ax = plt.subplots(figsize=(12,4))
df.plot(ax=ax)
ax.set_ylim(0, 1)

In [20]:
conflict_dyads = Counter()
conflict_models = defaultdict(set)

for i, model in enumerate(model_results):
#for i, model in enumerate(all_models):
    for event in model.log.get_events(action="Attack"):
        source = event.source
        target = event.target
        dyad = (source, target)
        conflict_dyads[dyad] += 1
        conflict_models[dyad].add(i)

conflict_models = {k: len(v) for k, v in conflict_models.items()}        
        
# Fill in zeroes
names = agents.Name.unique()
for source, target in permutations(names, 2):
    dyad = (source, target)
    if dyad not in conflict_dyads:
        conflict_dyads[dyad] = 0
    if dyad not in conflict_models:
        conflict_models[dyad] = 0

In [21]:
conflict_dyads

Counter({('Argentina', 'Taiwan'): 0,
         ('United States of America', 'Colombia'): 0,
         ('Iran', 'Algeria'): 0,
         ('Argentina', 'United Kingdom'): 0,
         ('Iran', 'Syria'): 0,
         ('Netherlands', 'Taiwan'): 0,
         ('Saudi Arabia', 'Ethiopia'): 0,
         ('Sweden', 'Thailand'): 0,
         ('France', 'Saudi Arabia'): 0,
         ('Brazil', 'Italy'): 0,
         ('India', 'Taiwan'): 0,
         ('Morocco', 'Colombia'): 0,
         ('Iraq', 'Ethiopia'): 0,
         ('Thailand', 'Poland'): 0,
         ('Pakistan', 'Belgium'): 0,
         ('South Korea', 'Belgium'): 0,
         ('Turkey', 'Morocco'): 0,
         ('Brazil', 'United Kingdom'): 0,
         ('Japan', 'Turkey'): 0,
         ('United States of America', 'Venezuela'): 0,
         ('Ukraine', 'Germany'): 0,
         ('Sweden', 'Saudi Arabia'): 0,
         ('Turkey', 'Iraq'): 0,
         ('Nigeria', 'Germany'): 0,
         ('Chile', 'Syria'): 0,
         ('Morocco', 'Iran'): 0,
         ('North Ko

## Get ICEWS Data

Data prepared in [6.05 ICEWS Aggregation](http://localhost:8888/notebooks/Programming/ConflictModel/negotiation_model/6.05%20ICEWS%20Aggregation.ipynb)

In [ ]:
icews = pd.read_csv("Negative_Dyads_2005-2006.csv")
icews = icews[["Source_Country", "Target_Country", "Event_ID"]]
icews.rename(columns={"Event_ID": "Count"}, inplace=True)

In [ ]:
icews.head()

In [ ]:
icews.replace("Democratic Republic of Congo", "Democratic Republic of the Congo", inplace=True)
icews.replace("Russian Federation", "Russia", inplace=True)
icews.replace("United States", "United States of America", inplace=True)

# Compare

In [ ]:
model_conflicts = pd.DataFrame({"Model_Conflicts": dict(conflict_dyads),
                               "Models_with_Conflicts": conflict_models})
model_conflicts.reset_index(inplace=True)
model_conflicts.rename(columns={"level_0": "Source_Country",
                               "level_1": "Target_Country"}, inplace=True)

In [ ]:
model_conflicts.head()

In [ ]:
df = model_conflicts.merge(icews, how="left", on=["Source_Country", "Target_Country"])
df.rename(columns={"Count": "ICEWS_Count"}, inplace=True)
df.fillna(0, inplace=True)

In [ ]:
df.head()

In [ ]:
import statsmodels.api as sm
import statsmodels
from statsmodels.discrete.discrete_model import NegativeBinomial
from statsmodels.discrete.discrete_model import Poisson
from statsmodels.miscmodels.count import PoissonZiGMLE

In [ ]:
df["Const."] = 1

In [ ]:
lm = sm.OLS(df["ICEWS_Count"], df[["Const.", "Model_Conflicts"]])
fit = lm.fit()

In [ ]:
print(fit.summary())

In [ ]:
nb = NegativeBinomial(df["ICEWS_Count"], df[["Const.", "Model_Conflicts"]])
fit = nb.fit()
print(fit.summary())

In [ ]:
ps = Poisson(df["ICEWS_Count"], df[["Const.", "Model_Conflicts"]])
fit = ps.fit_regularized()
print(fit.summary())

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
ax.scatter(df.Model_Conflicts, df.ICEWS_Count)
ax.set_xlabel("Model Conflicts")
ax.set_ylabel("ICEWS Conflicts")
#ax.set_xlim(-5, 100)

In [ ]:
df.ICEWS_Count.hist()
plt.yscale('log')

In [ ]:
df.corr(method="spearman")

In [ ]:
df.corr()

### At least N events

In [ ]:
df["ICEWS_Dummy"] = 0
df["ICEWS_Dummy"][df.ICEWS_Count > 5] = 1

In [ ]:
logit = sm.Logit(df["ICEWS_Dummy"], df[["Const.", "Model_Conflicts"]])
fit = logit.fit()
print(fit.summary())

In [ ]:
pd.crosstab(df.ICEWS_Dummy, df.Model_Conflicts)

In [ ]:
logit = sm.Logit(df["ICEWS_Dummy"], df[["Const.", "Models_with_Conflicts"]])
fit = logit.fit()
print(fit.summary())

In [ ]:
df["Model_Dummy"] = 0
df["Model_Dummy"][df.Model_Conflicts > 0] = 1

In [ ]:
logit = sm.Logit(df["ICEWS_Dummy"], df[["Const.", "Model_Dummy"]])
fit = logit.fit()
print(fit.summary())

In [ ]:
df["Model_Dummy"] = 0
df["Model_Dummy"][df.Models_with_Conflicts > 0] = 1

logit = sm.Logit(df["ICEWS_Dummy"], df[["Const.", "Model_Dummy"]])
fit = logit.fit()
print(fit.summary())

### Is it better than just using $\Delta$Taus?

In [ ]:
dyad_taus = {}
for i, row in df.iterrows():
    source = row.Source_Country
    target = row.Target_Country
    tau = get_tau(source, target)
    dyad_taus[(source, target)] = tau

In [ ]:
taus = pd.Series(dyad_taus).reset_index()
taus.rename(columns={"level_0": "Source_Country", 
                     "level_1": "Target_Country",
                     0: "Tau"}, inplace=True)

In [ ]:
df = df.merge(taus, how='left', on=["Source_Country", "Target_Country"])

In [ ]:
df["Tau_Dummy"] = 0
df["Tau_Dummy"][df.Tau != 0] = 1

In [ ]:
lm = sm.OLS(df["ICEWS_Count"], df[["Const.", "Tau"]])
fit = lm.fit()
print(fit.summary())

In [ ]:
logit = sm.Logit(df["ICEWS_Dummy"], df[["Const.", "Tau_Dummy", "Tau"]])
fit = logit.fit()
print(fit.summary())

In [ ]:
logit = sm.Logit(df["ICEWS_Dummy"], df[["Const.", "Tau", "Model_Conflicts"]])
fit = logit.fit()
print(fit.summary())

In [ ]:
ps = Poisson(df["ICEWS_Count"], df[["Const.", "Tau", "Model_Conflicts"]])
fit = ps.fit_regularized()
print(fit.summary())